In [4]:
import rosbag
input_bag =  "/home/xuhao/bags/swarm_local_2020_12/swarm_local_2020-12-10-21-06-07.bag"
output_bag = "/home/xuhao/bags/swarm_local_2020_12/swarm_local_2020-12-10-21-06-07_processed.bag"
moveout_topics = {
    "/swarm_drones/node_detected",
    "/swarm_drones/swarm_drone_fused",
    "/swarm_drones/swarm_drone_fused_relative",
    "/swarm_drones/swarm_frame",
    "/swarm_drones/swarm_frame_predict"
}
#Moveout node_detected
with rosbag.Bag(output_bag, 'w') as outbag:
    for topic, msg, t in rosbag.Bag(input_bag).read_messages():
        if topic in moveout_topics:
            pass
        else:
            #outbag.write(topic, msg, msg.header.stamp if msg._has_header else t)
            outbag.write(topic, msg, t)            